In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import random 
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import minmax_scale
from datetime import datetime
import re

In [77]:
data1 = pd.read_csv("train/subs_csi_train.csv", sep = ";")
data2 = pd.read_csv("train/subs_features_train.csv", sep = ";")

In [78]:
data1.head()

,SK_ID,CSI,CONTACT_DATE
0,2,0,4.05
1,3,0,21.05
2,4,0,27.05
3,5,0,8.05
4,6,1,6.05


In [79]:
data2.head()

,SNAP_DATE,COM_CAT#1,SK_ID,COM_CAT#2,COM_CAT#3,BASE_TYPE,ACT,ARPU_GROUP,COM_CAT#7,COM_CAT#8,...,COM_CAT#25,COM_CAT#26,COM_CAT#27,COM_CAT#28,COM_CAT#29,COM_CAT#30,COM_CAT#31,COM_CAT#32,COM_CAT#33,COM_CAT#34
0,01.08.01,4,3339,80,5,2,1,12.0,1,1489.0,...,1,1,",031696",",076724",",040279",",0133",",083382",",027579",",103896",2.0
1,01.08.01,4,238,75,5,2,1,7.0,1,945.0,...,1,1,",015893",",012981",0,0,",04893",",027579",",103896",3.0
2,01.08.01,4,4669,43,5,2,1,3.0,3,1033.0,...,1,1,",000415",0,0,0,",009397",",027579",",103896",3.0
3,01.08.01,4,2555,54,5,2,1,12.0,1,512.0,...,1,1,",04651",",039486",",000829",0,",196676",",027579",",103896",1.0
4,01.08.01,4,2291,38,5,2,1,7.0,1,1268.0,...,1,1,",005393",",007385",",003884",0,",047297",",027579",",103896",6.0


In [80]:
train = pd.merge(data1,data2,on='SK_ID',how='left')
print(train.shape)
train.head()

(47411, 38)


,SK_ID,CSI,CONTACT_DATE,SNAP_DATE,COM_CAT#1,COM_CAT#2,COM_CAT#3,BASE_TYPE,ACT,ARPU_GROUP,...,COM_CAT#25,COM_CAT#26,COM_CAT#27,COM_CAT#28,COM_CAT#29,COM_CAT#30,COM_CAT#31,COM_CAT#32,COM_CAT#33,COM_CAT#34
0,2,0,4.05,01.03.02,7,77,10,1,1,10.0,...,1,1,",00207",",007335",",00005",",002785",",029765",",027579",",103896",NaN
1,2,0,4.05,01.02.02,7,77,9,1,1,10.0,...,1,1,",003913",",00159",",000018",",001954",",054871",",027579",",103896",NaN
2,2,0,4.05,01.05.02,7,77,12,1,1,9.0,...,1,1,",001801",",008317",",000081",",002441",",018126",",027579",",103896",9.0
3,2,0,4.05,01.01.02,7,77,17,1,1,6.0,...,1,0,",001798",",002048",",000033",",002331",",022108",",027579",",103896",NaN
4,2,0,4.05,01.04.02,7,77,11,1,1,8.0,...,1,1,",00652",",001408",",00001",",001709",",022772",",027579",",103896",9.0


In [81]:
def to_f(x):
    if issubclass(str,type(x)):
        if x.find(',') != -1:
            x = x.replace(',','.')
            return float(x)
    return x
to_float = np.vectorize(to_f)

In [82]:
train.isnull().sum()

SK_ID                  0
CSI                    0
CONTACT_DATE           0
SNAP_DATE              0
COM_CAT#1              0
COM_CAT#2              0
COM_CAT#3              0
BASE_TYPE              0
ACT                    0
ARPU_GROUP            21
COM_CAT#7              0
COM_CAT#8            124
DEVICE_TYPE_ID       111
INTERNET_TYPE_ID     111
REVENUE                0
ITC                    0
VAS                    0
RENT_CHANNEL           0
ROAM                   0
COST                   0
COM_CAT#17             0
COM_CAT#18             0
COM_CAT#19             0
COM_CAT#20             0
COM_CAT#21             0
COM_CAT#22             0
COM_CAT#23             0
COM_CAT#24             0
COM_CAT#25             0
COM_CAT#26             0
COM_CAT#27             0
COM_CAT#28             0
COM_CAT#29             0
COM_CAT#30             0
COM_CAT#31             0
COM_CAT#32             0
COM_CAT#33             0
COM_CAT#34          1603
dtype: int64

In [83]:
train['CSI'].value_counts()

0    39982
1     7429
Name: CSI, dtype: int64

In [84]:
for col in train.columns:
    if col == 'SNAP_DATE':
        train[col] = train[col].fillna('01.01.01')
    else:
        train[col] = train[col].fillna(-1)
train.head()

,SK_ID,CSI,CONTACT_DATE,SNAP_DATE,COM_CAT#1,COM_CAT#2,COM_CAT#3,BASE_TYPE,ACT,ARPU_GROUP,...,COM_CAT#25,COM_CAT#26,COM_CAT#27,COM_CAT#28,COM_CAT#29,COM_CAT#30,COM_CAT#31,COM_CAT#32,COM_CAT#33,COM_CAT#34
0,2,0,4.05,01.03.02,7,77,10,1,1,10.0,...,1,1,",00207",",007335",",00005",",002785",",029765",",027579",",103896",-1.0
1,2,0,4.05,01.02.02,7,77,9,1,1,10.0,...,1,1,",003913",",00159",",000018",",001954",",054871",",027579",",103896",-1.0
2,2,0,4.05,01.05.02,7,77,12,1,1,9.0,...,1,1,",001801",",008317",",000081",",002441",",018126",",027579",",103896",9.0
3,2,0,4.05,01.01.02,7,77,17,1,1,6.0,...,1,0,",001798",",002048",",000033",",002331",",022108",",027579",",103896",-1.0
4,2,0,4.05,01.04.02,7,77,11,1,1,8.0,...,1,1,",00652",",001408",",00001",",001709",",022772",",027579",",103896",9.0


In [85]:
for i in train.columns:
    if train[i].dtypes == np.object:
        train[i] = to_float(train[i])

In [86]:
columns = list(set(train.columns.values) - set(['CSI','SK_ID','SNAP_DATE']))
print(columns)
X = train[columns].values
y = train['CSI'].values

['COM_CAT#32', 'INTERNET_TYPE_ID', 'ITC', 'COM_CAT#3', 'RENT_CHANNEL', 'ROAM', 'CONTACT_DATE', 'COM_CAT#30', 'COM_CAT#23', 'COM_CAT#34', 'REVENUE', 'COM_CAT#33', 'COM_CAT#2', 'COM_CAT#24', 'COM_CAT#19', 'COM_CAT#26', 'COM_CAT#22', 'COM_CAT#18', 'COM_CAT#20', 'COM_CAT#21', 'COM_CAT#29', 'ACT', 'COM_CAT#27', 'COM_CAT#28', 'COM_CAT#17', 'BASE_TYPE', 'COM_CAT#31', 'COM_CAT#25', 'VAS', 'COST', 'COM_CAT#1', 'COM_CAT#7', 'COM_CAT#8', 'DEVICE_TYPE_ID', 'ARPU_GROUP']


In [87]:
data1_t = pd.read_csv("test/subs_csi_test.csv", sep = ";")
print(data1_t.shape)
data2_t = pd.read_csv("test/subs_features_test.csv", sep = ";")

(5221, 2)


In [88]:
test = pd.merge(data1_t, data2_t, on='SK_ID', how='left')
print(test.shape)
for col in test.columns:
    if col == 'SNAP_DATE':
        test[col] = test[col].fillna('01.01.01')
    else:
        test[col] = test[col].fillna(-1)
for i in test.columns:
    if test[i].dtypes == np.object:
        test[i] = to_float(test[i])
columns = list(set(test.columns.values) - set(['SK_ID', "SNAP_DATE"]))
print(columns)
X_t = test[columns].values

(58989, 37)
['COM_CAT#32', 'INTERNET_TYPE_ID', 'ITC', 'COM_CAT#3', 'RENT_CHANNEL', 'ROAM', 'CONTACT_DATE', 'COM_CAT#30', 'COM_CAT#23', 'COM_CAT#34', 'REVENUE', 'COM_CAT#33', 'COM_CAT#2', 'COM_CAT#24', 'COM_CAT#19', 'COM_CAT#26', 'COM_CAT#22', 'COM_CAT#18', 'COM_CAT#20', 'COM_CAT#21', 'COM_CAT#29', 'ACT', 'COM_CAT#27', 'COM_CAT#28', 'COM_CAT#17', 'BASE_TYPE', 'COM_CAT#31', 'COM_CAT#25', 'VAS', 'COST', 'COM_CAT#1', 'COM_CAT#7', 'COM_CAT#8', 'DEVICE_TYPE_ID', 'ARPU_GROUP']


In [147]:
model = CatBoostClassifier(iterations=2, depth=2, learning_rate=1, loss_function='Logloss', logging_level='Verbose')
model.fit(X,y)
ans = model.predict(X_t)

0:	learn: 0.4332786	total: 105ms	remaining: 105ms
1:	learn: 0.4302588	total: 145ms	remaining: 0us


In [148]:
df = pd.DataFrame(ans)
df.head()

,0
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [149]:
#df.to_csv("ans.csv",index=False,sep='\n')

In [150]:
dfd = test.copy()
dfd['RES'] = df
dfd['RES'].value_counts()

0.0    58989
Name: RES, dtype: int64

In [110]:
a = set()
with open("ans.csv","w") as file:
    for i in data1_t['SK_ID']:
        #if ((dfd[dfd['SK_ID'] == i]['RES'].value_counts()).shape[0] > 1):
        #    print('1\n',file=file,flush=True)
        #else:
           print('0\n',file=file,flush=True)

0


In [111]:
kek = pd.read_csv("ans.csv")
kek['0'].value_counts()

0    2653
1    2567
Name: 0, dtype: int64

In [95]:

for i in data1_t['SK_ID']:
    a.add(i)
print(len(a))

5221


In [114]:
print(kek.shape)

(5220, 1)


In [134]:
a = [i%2 for i in range(5221)]

In [135]:
len(a)

5221

In [145]:
a = [i%2 for i in range(5221)]
random.shuffle(a)
random.shuffle(a)
random.shuffle(a)
random.shuffle(a)
random.shuffle(a)
random.shuffle(a)
random.shuffle(a)
random.shuffle(a)
random.shuffle(a)
random.shuffle(a)
random.shuffle(a)
random.shuffle(a)
random.shuffle(a)
random.shuffle(a)
random.shuffle(a)
random.shuffle(a)
random.shuffle(a)
random.shuffle(a)
random.shuffle(a)
random.shuffle(a)
random.shuffle(a)
random.shuffle(a)
random.shuffle(a)
with open("kek","w") as file:
    for i in a:
        print(i,file=file)

In [144]:
with open("kek","w") as file:
    for i in a:
        print(i,file=file)

In [ ]:
select * from subs_csi JOIN subs_bs_consumption
